In [2]:
import pandas as pd
import numpy as np
from pandas import json_normalize
from fastapi import FastAPI,Body

In [3]:
# Para Funcion 1
movies_df_func1 = pd.read_csv("ETL_movies_languages.csv")

In [4]:
movies_df_func1 = movies_df_func1[["Movie_id","short_language"]]
movies_df_func1.to_csv("movies_df_func1.csv",index=False)

In [5]:
movies_df_func1.loc[movies_df_func1["short_language"]=="en",["short_language"]].count()

short_language    23981
dtype: int64

In [6]:
# Se ingresa un idioma (como están escritos en el dataset, no hay que traducirlos!).
# Debe devolver la cantidad de películas producidas en ese idioma.

In [7]:
def peliculas_idioma(idioma: str):
    if idioma in movies_df_func1["short_language"].unique():
        seleccion = movies_df_func1.loc[movies_df_func1["short_language"] == idioma, ["short_language"]]
        respuesta = seleccion.shape[0]
        return {"message": f"La cantidad de Peliculas en el idioma {idioma} es {respuesta}"}
    else:
        return {"message": "El Idioma no se encuentra en la lista o El dato ingresado no es correcto, intenta escribir de la siguiente forma -> Ej: 'English'='en'"}


In [8]:
peliculas_idioma("en")

{'message': 'La cantidad de Peliculas en el idioma en es 23981'}

In [6]:
# Para Funcion 2
movies_df_func2 = pd.read_csv("ETL_movies_main.csv")

In [7]:
movies_df_func2 = movies_df_func2.loc[:,["title","runtime","release_year"]]
movies_df_func2.to_csv("movies_df_func2.csv",index=False)

In [ ]:
#Se ingresa una pelicula. Debe devolver la duracion y el año.

In [8]:
def peliculas_duracion(movie_title: str):
    if movie_title in movies_df_func2["title"].values:
        seleccion = movies_df_func2[movies_df_func2["title"] == movie_title]
        runtime_ = seleccion.loc[seleccion.index[0], "runtime"]
        release_year_ = seleccion.loc[seleccion.index[0], "release_year"]
        return {
            "message": f"La película {movie_title} tiene una duración de {runtime_} minutos y se estrenó en el año {release_year_}"
        }
    else:
        return {
            "message": "La película no está en la lista o intenta escribir el nombre de la siguiente forma -> Ej: 'Toy Story'"
        }


In [9]:
peliculas_duracion("Star Wars")

{'message': 'La película Star Wars tiene una duración de 121.0 minutos y se estrenó en el año 1977'}

In [10]:
# para Funcion 3
#Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio

In [146]:
movies_df_func3_collections = pd.read_csv("ETL_movies_collection.csv")
movies_df_func3_collections = movies_df_func3_collections[["Movie_id","collection_name"]]
movies_df_func3_collections

,Movie_id,collection_name
0,862,Toy Story Collection
1,15602,Grumpy Old Men Collection
2,11862,Father of the Bride Collection
3,710,James Bond Collection
4,21032,Balto Collection
...,...,...
4480,24568,The Carry On Collection
4481,19307,The Carry On Collection
4482,21251,The Carry On Collection
4483,460135,DC Super Hero Girls Collection


In [147]:
movies_df_func3_main = pd.read_csv("ETL_movies_main.csv")
movies_df_func3_main = movies_df_func3_main[["Movie_id","revenue"]]
movies_df_func3_main

,Movie_id,revenue
0,862,373554033.0
1,8844,262797249.0
2,15602,0.0
3,31357,81452156.0
4,11862,76578911.0
...,...,...
45341,30840,0.0
45342,111109,0.0
45343,67758,0.0
45344,227506,0.0


In [148]:
movies_df_func3 = movies_df_func3_main.merge(movies_df_func3_collections,how="left",left_on="Movie_id",right_on="Movie_id")
movies_df_func3 = movies_df_func3.dropna(subset=["collection_name"],axis=0)
movies_df_func3.to_csv("movies_df_func3.csv",index=False)

In [149]:
movies_df_func3

,Movie_id,revenue,collection_name
0,862,373554033.0,Toy Story Collection
2,15602,0.0,Grumpy Old Men Collection
4,11862,76578911.0,Father of the Bride Collection
9,710,352194034.0,James Bond Collection
12,21032,11348324.0,Balto Collection
...,...,...,...
45237,24568,0.0,The Carry On Collection
45240,19307,0.0,The Carry On Collection
45251,21251,0.0,The Carry On Collection
45253,460135,0.0,DC Super Hero Girls Collection


In [150]:
def franquicia(nombre_franquicia: str):
    if nombre_franquicia in movies_df_func3["collection_name"].unique():
        filtro = movies_df_func3.loc[movies_df_func3["collection_name"] == nombre_franquicia]
        cantidad_pelis = filtro["Movie_id"].count()
        ganancia_total = filtro["revenue"].sum()
        ganancia_promedio = filtro["revenue"].mean()
        
        ganancia_total_formatted = "{:.2f}".format(ganancia_total)
        ganancia_promedio_formatted = "{:.2f}".format(ganancia_promedio)
        
        return f"La franquicia {nombre_franquicia} cuenta con {cantidad_pelis} peliculas, produjo una ganancia Bruta de {ganancia_total_formatted} y una ganancia Bruta promedio por pelicula de {ganancia_promedio_formatted}"
    else:
        return "La franquicia no existe en el catalogo o prueba escribiendo de la siguiente forma -> Ej: 'Toy Story Collection' "


In [151]:
franquicia("Toy Story Collection")

'La franquicia Toy Story Collection cuenta con 3 peliculas, produjo una ganancia Bruta de 1937890605.00 y una ganancia Bruta promedio por pelicula de 645963535.00'

In [17]:
# para Funcion 4
#Se ingresa un país (como están escritos en el dataset, no hay que traducirlos!), retornando la cantidad de peliculas producidas en el mismo.


In [18]:
movies_df_func4 = pd.read_csv("ETL_movies_productions.csv")
movies_df_func4 = movies_df_func4[["Movie_id","country_code","country_name"]]
movies_df_func4.to_csv("movies_df_func4.csv",index=False)
movies_df_func4

,Movie_id,country_code,country_name
0,862,US,United States of America
1,8844,US,United States of America
2,15602,US,United States of America
3,31357,US,United States of America
4,11862,US,United States of America
...,...,...,...
55556,30840,US,United States of America
55557,111109,PH,Philippines
55558,67758,US,United States of America
55559,227506,RU,Russia


In [19]:
def peliculas_pais(nombre_pais: str):
    if nombre_pais in movies_df_func4["country_name"].unique():
        filtro = movies_df_func4.loc[movies_df_func4["country_name"] == nombre_pais]
        cantidad_pelis_producidas = filtro["Movie_id"].count()
        return f"La cantidad de Peliculas producidas en {nombre_pais} es de {cantidad_pelis_producidas}"
    else:
        return "Nombre de Pais no esta en la lista o error de escritura intenta escribirlo de la siguiente forma -> Ej: 'Germany' "

In [20]:
peliculas_pais("Russia")

'La cantidad de Peliculas producidas en Russia es de 911'

In [21]:
# para Funcion 5
#Se ingresa la productora, entregandote el revunue total y la cantidad de peliculas que realizo.

In [27]:
movies_df_func5_productora = pd.read_csv("ETL_movies_productions.csv")
movies_df_func5_productora = movies_df_func5_productora.loc[:,["Movie_id","company_name"]]
movies_df_func5_productora = movies_df_func5_productora.drop_duplicates(subset=["Movie_id"],keep="first")
movies_df_func5_productora

,Movie_id,company_name
0,862,Pixar Animation Studios
1,8844,TriStar Pictures
2,15602,Teitler Film
3,31357,Interscope Communications
4,11862,Warner Bros.
...,...,...
55553,30840,Trimark Pictures
55557,111109,Haut et Court
55558,67758,Mille et Une Productions
55559,227506,Propaganda Pictures


In [28]:
movies_df_func5_main = pd.read_csv("ETL_movies_main.csv")
movies_df_func5_main = movies_df_func5_main.loc[:,["Movie_id","revenue"]]
movies_df_func5_main

,Movie_id,revenue
0,862,373554033.0
1,8844,262797249.0
2,15602,0.0
3,31357,81452156.0
4,11862,76578911.0
...,...,...
45341,30840,0.0
45342,111109,0.0
45343,67758,0.0
45344,227506,0.0


In [29]:
movies_df_func5 = movies_df_func5_main.merge(movies_df_func5_productora,how="left",left_on="Movie_id",right_on="Movie_id")
movies_df_func5 = movies_df_func5.dropna(subset=["company_name"],axis=0)
movies_df_func5.to_csv("movies_df_func5.csv",index=False)
movies_df_func5

,Movie_id,revenue,company_name
0,862,373554033.0,Pixar Animation Studios
1,8844,262797249.0,TriStar Pictures
2,15602,0.0,Teitler Film
3,31357,81452156.0,Interscope Communications
4,11862,76578911.0,Warner Bros.
...,...,...,...
45341,30840,0.0,Trimark Pictures
45342,111109,0.0,Haut et Court
45343,67758,0.0,Mille et Une Productions
45344,227506,0.0,Propaganda Pictures


In [30]:
def productoras_exitosas(nombre_productora: str):
    if nombre_productora in movies_df_func5["company_name"].unique():
        filtro = movies_df_func5.loc[movies_df_func5["company_name"] == nombre_productora]
        ganancia_productora = filtro["revenue"].sum()
        cantidad_pelis_productora = filtro["Movie_id"].count()

        ganancia_productora_r = round(ganancia_productora, 2)
        return f"La productora {nombre_productora} produjo una ganancia total Bruta de {ganancia_productora_r} y realizo una cantidad de {cantidad_pelis_productora} peliculas"
    else:
        return "Nombre de Productora no esta en la lista o error de escritura intenta escribirlo de la siguiente forma -> Ej: 'Pixar Animation Studios' "
    

In [31]:
productoras_exitosas("Interscope Communications")

'La productora Interscope Communications produjo una ganancia total Bruta de 1729655632.0 y realizo una cantidad de 34 peliculas'

In [117]:
# para Funcion 6
#Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno.
#Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma,
# en formato lista.

In [36]:
movies_df_func6_director = pd.read_csv("ETL_credits_crew.csv")
movies_df_func6_director = movies_df_func6_director.loc[:,["Movie_id","name","job"]]
movies_df_func6_director = movies_df_func6_director.loc[movies_df_func6_director["job"]=="Director"]
movies_df_func6_director = movies_df_func6_director.loc[:,["Movie_id","name"]]
movies_df_func6_director = movies_df_func6_director.drop_duplicates(subset=["Movie_id"],keep="first")
movies_df_func6_director

,Movie_id,name
0,862,John Lasseter
109,8844,Joe Johnston
122,15602,Howard Deutch
126,31357,Forest Whitaker
141,11862,Charles Shyer
...,...,...
464675,439050,Hamid Nematollah
464684,111109,Lav Diaz
464690,67758,Mark L. Lester
464695,227506,Yakov Protazanov


In [38]:
movies_df_func6_main = pd.read_csv("ETL_movies_main.csv")
movies_df_func6_main = movies_df_func6_main.loc[:,["Movie_id","title","release_date","budget","revenue","return"]]
movies_df_func6_main

,Movie_id,title,release_date,budget,revenue,return
0,862,Toy Story,1995-10-30,30000000.0,373554033.0,12.451801
1,8844,Jumanji,1995-12-15,65000000.0,262797249.0,4.043035
2,15602,Grumpier Old Men,1995-12-22,0.0,0.0,0.000000
3,31357,Waiting to Exhale,1995-12-22,16000000.0,81452156.0,5.090760
4,11862,Father of the Bride Part II,1995-02-10,0.0,76578911.0,0.000000
...,...,...,...,...,...,...
45341,30840,Robin Hood,1991-05-13,0.0,0.0,0.000000
45342,111109,Century of Birthing,2011-11-17,0.0,0.0,0.000000
45343,67758,Betrayal,2003-08-01,0.0,0.0,0.000000
45344,227506,Satan Triumphant,1917-10-21,0.0,0.0,0.000000


In [39]:
movies_df_func6 = movies_df_func6_main.merge(movies_df_func6_director,how="left",left_on="Movie_id",right_on="Movie_id")
movies_df_func6.to_csv("movies_df_func6.csv",index=False)
movies_df_func6

,Movie_id,title,release_date,budget,revenue,return,name
0,862,Toy Story,1995-10-30,30000000.0,373554033.0,12.451801,John Lasseter
1,8844,Jumanji,1995-12-15,65000000.0,262797249.0,4.043035,Joe Johnston
2,15602,Grumpier Old Men,1995-12-22,0.0,0.0,0.000000,Howard Deutch
3,31357,Waiting to Exhale,1995-12-22,16000000.0,81452156.0,5.090760,Forest Whitaker
4,11862,Father of the Bride Part II,1995-02-10,0.0,76578911.0,0.000000,Charles Shyer
...,...,...,...,...,...,...,...
45341,30840,Robin Hood,1991-05-13,0.0,0.0,0.000000,John Irvin
45342,111109,Century of Birthing,2011-11-17,0.0,0.0,0.000000,Lav Diaz
45343,67758,Betrayal,2003-08-01,0.0,0.0,0.000000,Mark L. Lester
45344,227506,Satan Triumphant,1917-10-21,0.0,0.0,0.000000,Yakov Protazanov


In [40]:
def get_director(nombre_director):
    if nombre_director in movies_df_func6["name"].unique():
        director_movies = movies_df_func6[movies_df_func6["name"] == nombre_director]
        total_return = director_movies["return"].sum()
        
        movies_info = []
        for index, row in director_movies.iterrows():
            movie_info = {
                "Nombre Pelicula": row["title"],
                "Fecha de Estreno": row["release_date"],
                "Ganancia Bruta": row["return"],
                "Presupuesto": row["budget"],
                "Ganancia Neta": row["revenue"]
            }
            movies_info.append(movie_info)
        
        return {
            "Exito,Retorno Total": total_return,
            "Peliculas": movies_info
        }
    else:
        return "Director no esta en la lista o intenta escribir de la siguiente manera -> Ej:'George Lucas' "

In [41]:
get_director("George Lucas")

{'Exito,Retorno Total': 274.77865083544054,
 'Peliculas': [{'Nombre Pelicula': 'Star Wars',
   'Fecha de Estreno': '1977-05-25',
   'Ganancia Bruta': 70.49072790909091,
   'Presupuesto': 11000000.0,
   'Ganancia Neta': 775398007.0},
  {'Nombre Pelicula': 'Star Wars: Episode I - The Phantom Menace',
   'Fecha de Estreno': '1999-05-19',
   'Ganancia Bruta': 8.037543982608696,
   'Presupuesto': 115000000.0,
   'Ganancia Neta': 924317558.0},
  {'Nombre Pelicula': 'American Graffiti',
   'Fecha de Estreno': '1973-08-01',
   'Ganancia Bruta': 180.18018018018017,
   'Presupuesto': 777000.0,
   'Ganancia Neta': 140000000.0},
  {'Nombre Pelicula': 'Star Wars: Episode II - Attack of the Clones',
   'Fecha de Estreno': '2002-05-15',
   'Ganancia Bruta': 5.411652733333334,
   'Presupuesto': 120000000.0,
   'Ganancia Neta': 649398328.0},
  {'Nombre Pelicula': 'THX 1138',
   'Fecha de Estreno': '1971-03-11',
   'Ganancia Bruta': 3.1364221364221363,
   'Presupuesto': 777000.0,
   'Ganancia Neta': 243

In [ ]:
# ML,Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

In [49]:
df_ml_main = pd.read_csv("ETL_movies_main.csv")
df_ml_main = df_ml_main[["Movie_id","popularity","title"]]
df_ml_main["popularity_fixed"] = df_ml_main["popularity"].apply(lambda x: x / 1000000 if x > 0 else x)
df_ml_main

,Movie_id,popularity,title,popularity_fixed
0,862,30000000.0,Toy Story,30.0
1,8844,65000000.0,Jumanji,65.0
2,15602,0.0,Grumpier Old Men,0.0
3,31357,16000000.0,Waiting to Exhale,16.0
4,11862,0.0,Father of the Bride Part II,0.0
...,...,...,...,...
45341,30840,0.0,Robin Hood,0.0
45342,111109,0.0,Century of Birthing,0.0
45343,67758,0.0,Betrayal,0.0
45344,227506,0.0,Satan Triumphant,0.0


In [51]:
df_ml_genres_grouped = pd.read_csv("ETL_movies_genres_grouped.csv")
df_ml_genres_grouped

,Movie_id,genero
0,2,Drama Crime
1,3,Drama Comedy
2,5,Crime Comedy
3,6,Action Thriller Crime
4,11,Adventure Action Science Fiction
...,...,...
42957,465044,Fantasy Drama
42958,467731,Drama
42959,468343,Drama Romance
42960,468707,Romance Comedy


In [80]:
df_ml = df_ml_main.merge(df_ml_genres_grouped,how="left",left_on="Movie_id",right_on="Movie_id")
df_ml = df_ml.loc[:,["title","popularity_fixed","genero"]]
df_ml

,title,popularity_fixed,genero
0,Toy Story,30.0,Animation Comedy Family
1,Jumanji,65.0,Adventure Fantasy Family
2,Grumpier Old Men,0.0,Romance Comedy
3,Waiting to Exhale,16.0,Comedy Drama Romance
4,Father of the Bride Part II,0.0,Comedy
...,...,...,...
45341,Robin Hood,0.0,Drama Action Romance
45342,Century of Birthing,0.0,Drama
45343,Betrayal,0.0,Action Drama Thriller
45344,Satan Triumphant,0.0,NaN


In [81]:
# I Splitted the 'genero' column by spaces and create new columns
genre_columns = df_ml['genero'].str.split(expand=True)
genre_columns.columns = [f'genero{i+1}' for i in range(len(genre_columns.columns))]

# I Concatenated the new genre columns with the original DataFrame
df = pd.concat([df_ml, genre_columns], axis=1)

# Drop the original 'genero' column
df = df.drop('genero', axis=1)
df_ml = df.iloc[:,[0,1,2,3,4]]

In [82]:
df_ml

,title,popularity_fixed,genero1,genero2,genero3
0,Toy Story,30.0,Animation,Comedy,Family
1,Jumanji,65.0,Adventure,Fantasy,Family
2,Grumpier Old Men,0.0,Romance,Comedy,None
3,Waiting to Exhale,16.0,Comedy,Drama,Romance
4,Father of the Bride Part II,0.0,Comedy,None,None
...,...,...,...,...,...
45341,Robin Hood,0.0,Drama,Action,Romance
45342,Century of Birthing,0.0,Drama,None,None
45343,Betrayal,0.0,Action,Drama,Thriller
45344,Satan Triumphant,0.0,NaN,NaN,NaN


In [138]:
#exportando a csv
df_ml.to_csv("df_ml.csv",index=False)

In [ ]:
# Trying DB-Scan unsupervised learning model like we sow in classes

In [119]:
# este modelo me dio masomenos

In [86]:
# Usando dataframe mas Pequeño
df_ml = df_ml.iloc[:10000]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_ml['title'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# clustering
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
df_ml['cluster'] = dbscan.fit_predict(cosine_sim)



# Funcion
#@app.get("/main/")
def recomendacion(title: str):
    num_recommendations=5
    input_index = df_ml[df_ml['title'] == title].index[0]
    cluster_label = df_ml.loc[input_index, 'cluster']
    
    cluster_indices = df_ml[df_ml['cluster'] == cluster_label].index
    cluster_indices = cluster_indices[cluster_indices != input_index]  # Exclude input index
    
    similar_titles = df_ml.loc[cluster_indices, 'title'].head(num_recommendations).tolist()
    return similar_titles


In [118]:
recomendacion("Batman")

['Toy Story',
 'Grumpier Old Men',
 'Father of the Bride Part II',
 'Heat',
 'Tom and Huck']

In [ ]:
# Trying DB-cosine_similarity model 

In [120]:
# Este anda bien, pero me falta solucionar que me tira repetidas las pelis

In [121]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


# Combine the genre columns into a single column
df_ml['genres'] = df_ml[['genero1', 'genero2', 'genero3']].apply(lambda row: ' '.join(row.dropna()), axis=1)

# Calculate the cosine similarity between genre vectors
vectorizer = CountVectorizer().fit_transform(df_ml['genres'])
genre_similarity = cosine_similarity(vectorizer)

# Function to find similar movies
def find_similar_movies(movie_title, top_n=5):
    movie_idx = df_ml[df_ml['title'] == movie_title].index[0]
    similar_movies_indices = genre_similarity[movie_idx].argsort()[-top_n-1:-1][::-1]  # Exclude the input movie itself
    similar_movie_titles = [df_ml['title'][idx] for idx in similar_movies_indices]

    return similar_movie_titles


In [122]:
find_similar_movies("Jumanji")

['The Princess Bride',
 'Jumanji',
 'Labyrinth',
 'The Wizard of Oz',
 'Return to Oz']

In [123]:
# Este anda bien y solucione lo de las pelis repetidas

In [136]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Combinando las columnas de genero en una
df_ml['genres'] = df_ml[['genero1', 'genero2', 'genero3']].apply(lambda row: ' '.join(row.dropna()), axis=1)

# Calcular la similitud
vectorizer = CountVectorizer().fit_transform(df_ml['genres'])
genre_similarity = cosine_similarity(vectorizer)

# Funcion Nico
def recomendacion(movie_title, top_n=5):
    if movie_title not in df_ml['title'].values:
        return f"La pelicula ingresada no esta en el catalogo o esta mal escrita. Intenta escribiendo de la siguiente manera -> Ej: 'Star Wars'"
    
    movie_idx = df_ml[df_ml['title'] == movie_title].index[0]
    similar_movies_indices = genre_similarity[movie_idx].argsort()[-top_n-1:][::-1]  # Include the input movie itself
    similar_movie_titles = [df_ml['title'][idx] for idx in similar_movies_indices if df_ml['title'][idx] != movie_title]

    # Para que me devuelva siempre 5 Pelis
    if len(similar_movie_titles) < top_n:
        additional_movies = df_ml[~df_ml['title'].isin([movie_title] + similar_movie_titles)]['title'].sample(top_n - len(similar_movie_titles))
        similar_movie_titles.extend(additional_movies)
    
    return similar_movie_titles

In [137]:
find_similar_movies("Superman")

['Kull the Conqueror',
 'The Fifth Element',
 'The Borrowers',
 'Last Action Hero',
 'Superman II']

In [135]:
# Explicacion
# Similitud de Cosenos evalúa la similitud entre dos vectores distintos a 0 de un espacio de producto interno.
# En el contexto de encontrar películas similares basadas en sus géneros, Similitud de cosenos funciona bien porque
# trata a cada género como una dimensión en un espacio de alta dimensión, mide el ángulo entre dos vectores en lugar de su magnitud,
# lo que lo hace adecuado para casos en los que las características pueden tener escalas variables.
# Mediante el uso de vectores de frecuencia de género (CountVectorizer), se puede comparar las películas por géneros comunes, que es una medida de similitud.